# **Observability** means being able to know what's happening at any given time. By knowing what's happening within the kernel, it's easier for you to determine how to manage the non-determinism that's intrinsic to LLMs.

The best way to experience the observability that's built into Semantic Kernel is to use the [Azure Application Insights example](https://github.com/microsoft/semantic-kernel/tree/main/dotnet/samples/TelemetryExample), but if at the programming level you want to see what we've made easy for you to observe, then keep on going!

## 🔥 We warm up a kernel

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.1.0"

In [ ]:
// Load settings
#!import config/Settings.cs 

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

In [ ]:
using System;
using System.ComponentModel;
using System.Threading.Tasks;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;

Kernel kernel;

if (useAzureOpenAI) {
    kernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey)
        .Build();
} else {
    kernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(model, apiKey, orgId)
        .Build();
}

## 🔌 Then we add a Plugin

In [ ]:
// Make a plugin that can be used to get the current time
public class TimeInformationPlugin
{
    [KernelFunction]
    [Description("Retrieves the current time in UTC.")]
    public string GetCurrentUtcTime() => DateTime.UtcNow.ToString("R");
}
kernel.ImportPluginFromType<TimeInformationPlugin>();

## 🪝 And then we attach event handlers to report what's up

In [ ]:
using System.Text.Json;

JsonSerializerOptions s_jsonOptionsCache = new() { WriteIndented = true };

#pragma warning disable SKEXP0004

// Handler which is called before a function is invoked
void MyInvokingHandler(object? sender, FunctionInvokingEventArgs e)
{
    var func = e.Function.Metadata;
    Console.WriteLine($">> 👀 Invoking:\n  {func.PluginName}-{func.Name}");
}

// Handler which is called after a prompt is rendered
void MyRenderedHandler(object? sender, PromptRenderedEventArgs e)
{
    Console.WriteLine($">> 👀 Prompt sent to model:\n  {e.RenderedPrompt}");
}

// Handler which is called after a function is invoked
void MyInvokedHandler(object? sender, FunctionInvokedEventArgs e)
{
    if (e.Result.Metadata is not null && e.Result.Metadata.ContainsKey("Usage"))
    {
        string s = JsonSerializer.Serialize(e.Result.Metadata?["Usage"], s_jsonOptionsCache);
        Console.WriteLine($">> 👀 Token usage:{s}");
    }
}

// Add the handlers to the kernel
kernel.FunctionInvoking += MyInvokingHandler;
kernel.PromptRendered += MyRenderedHandler;
kernel.FunctionInvoked += MyInvokedHandler;

## 👀🏃 We can now clearly see what's happening along the way

In [ ]:
// Invoke the kernel with a prompt and allow the AI to automatically invoke functions
OpenAIPromptExecutionSettings settings = new() { ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions };
Console.WriteLine(await kernel.InvokePromptAsync("How many days until Christmas? Explain your thinking.", new(settings)))